# Koog



[Koog](https://docs.koog.ai/) a multiplatform library for building AI empowered applications in idiomatic Kotlin (in a way that feels natural for Kotlin devs).
It supports state of the art features like agents, MCP, tools, embeddings, and more.
It can currently target the JVM, WASM and JS.

Since it supports the JVM, we can use it in a Kotlin notebook.
Let's start by adding the dependency.

In [22]:
%use koog(0.3.0)

## Single-run agents

Create an agent for different providers

In [23]:
val systemPrompt = "You are a helpful assistant. Answer user questions concisely."

fun getOpenAiAgent() = AIAgent(
    executor = simpleOpenAIExecutor(System.getenv("OPENAI_API_KEY")),
    systemPrompt = systemPrompt,
    llmModel = OpenAIModels.Chat.GPT4o
)

fun getGoogleAiAgent() = AIAgent(
    executor = simpleGoogleAIExecutor(System.getenv("GOOGLEAI_API_KEY")),
    systemPrompt = systemPrompt,
    llmModel = GoogleModels.Gemini2_0FlashLite
)

Uncomment the agent you want to use

In [24]:
//val agent = getOpenAiAgent()
val agent = getGoogleAiAgent()

In [25]:
import kotlinx.coroutines.runBlocking

runBlocking {
    val result = agent.run("Hello! How can you help me?")
    println(result)
}

I can provide information, answer questions, and complete tasks as instructed. What can I do for you today?



## Tool calling

First, define the toolset

In [31]:
import ai.koog.agents.core.tools.annotations.Tool

@LLMDescription("Tools for performing basic arithmetic operations")
class CircleGeometryToolset : ToolSet {
    @Tool
    @LLMDescription("Circle are calculation")
    fun area(
        @LLMDescription("Circle circumference radius (floating point value)")
        r: Double,
    ) = r * Math.PI * Math.PI

    @Tool
    @LLMDescription("Circle circumference calculation")
    fun circumference(
        @LLMDescription("Circle circumference radius (floating point value)")
        r: Double,
    ) = r * 2 * Math.PI
}

// Add the tool to the tool registry
val toolRegistry = ToolRegistry {
    tools(CircleGeometryToolset())
}

Add an event handler feature to log tool calls

In [29]:
import ai.koog.agents.features.eventHandler.feature.EventHandler

val features: AIAgent.FeatureContext.() -> Unit = {
    install(EventHandler) {
        onToolCall { println("tool called $it") }
    }
}

Define the agent

In [32]:
val circleGeomertrySystemPrompt =
    "You are a helpful assistant that can perform circle area and circumference calculations using the provided tools."

fun getGoogleAiAgentWithTools() = AIAgent(
    executor = simpleGoogleAIExecutor(System.getenv("GOOGLEAI_API_KEY")),
    llmModel = GoogleModels.Gemini2_0FlashLite,
    toolRegistry = toolRegistry,
    systemPrompt = circleGeomertrySystemPrompt,
    installFeatures = features
)

fun getOpenAiAgentWithTools() = AIAgent(
    executor = simpleOpenAIExecutor(System.getenv("OPENAI_API_KEY")),
    llmModel = OpenAIModels.Chat.GPT4o,
    systemPrompt = circleGeomertrySystemPrompt,
    toolRegistry = toolRegistry,
    installFeatures = features
)

val agentWithTools = getGoogleAiAgentWithTools()
//val agentWithTools = getOpenAiAgentWithTools()

In [33]:
import kotlinx.coroutines.runBlocking

runBlocking {
    val result = agentWithTools.run("What is the are and circumference of a circle with radius 5?")
    println(result)
    println("Cecck area: ${(5 * Math.PI * Math.PI)} Circumference: ${5 * 2 * Math.PI}")
}

tool called ToolCallContext(runId=c7ba42d3-cbb5-4240-bdde-221b44bac597, toolCallId=ff96ff8b-d21d-42ee-9db3-495a7f3a28f8, tool=ai.koog.agents.core.tools.reflect.ToolFromCallable@162aa5e1, toolArgs=VarArgs(args={parameter #1 r of fun Line_32_jupyter.CircleGeometryToolset.area(kotlin.Double): kotlin.Double=5.0}))
tool called ToolCallContext(runId=c7ba42d3-cbb5-4240-bdde-221b44bac597, toolCallId=a5e361f8-27ef-447a-99e9-e44ae2445f9a, tool=ai.koog.agents.core.tools.reflect.ToolFromCallable@3ee70f51, toolArgs=VarArgs(args={parameter #1 r of fun Line_32_jupyter.CircleGeometryToolset.circumference(kotlin.Double): kotlin.Double=5.0}))
The area is 49.348 and the circumference is 31.416.

Cecck area: 49.348022005446786 Circumference: 31.41592653589793
